In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
import plotly.graph_objects as go # for visualization
from sklearn.linear_model import LinearRegression

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [ ]:
mod0 = pd.read_excel("../../../../mod0.xlsx")
mod1 = pd.read_excel("../../../../mod1.xlsx")
mod2 = pd.read_excel("../../../../mod2.xlsx")
mod3 = pd.read_excel("../../../../mod3.xlsx")
mod4 = pd.read_excel("../../../../mod4.xlsx")
mod5 = pd.read_excel("../../../../mod5.xlsx")
mod6 = pd.read_excel("../../../../mod6.xlsx")
mod7 = pd.read_excel("../../../../mod7.xlsx")
mod8 = pd.read_excel("../../../../mod8.xlsx")
mod0.head(1)

In [ ]:
# X = mod0.drop(["Israelis_Count","Tourists_Count","Total"],axis=1)
# y_isrealis=mod0.Israelis_Count
# y_tourists=mod0.Tourists_Count
# y_total=mod0.Total


In [ ]:
# c=[]
# b=False
# for i in range(len(X.columns)):
#     if X.nunique()[i]<=2 and b:
#         c.append(X.columns[i])
#     if X.columns[i]=='visit_duration':
#         b=True

# c
# X=X.drop(c,axis=1)

In [ ]:

# m=mod0.drop(["Unnamed: 0",'Unnamed: 0.1',"Date","Model_number","Site_Name"],axis=1)
# X = m.drop(["Israelis_Count","Tourists_Count","Total"],axis=1)
# y_isrealis=m.Israelis_Count
# y_tourists=m.Tourists_Count
# y_total=m.Total
# targets_list=[y_isrealis,y_tourists,y_total]
# for y in targets_list:
#     f_reg_selector = SelectKBest(f_regression, k=10)
#     f_reg_selector.fit(X, y)
#     f_reg_support = f_reg_selector.get_support()
#     f_reg_feature = X.loc[:,f_reg_support].columns.to_list()
#     print("The top 6 top scored features are ",f_reg_feature)

# str(f_reg_feature)

In [12]:
import warnings
warnings.filterwarnings("ignore")

Model_type= "MLR"
res=pd.read_excel("../../../../res.xlsx")



mods=[mod0,mod1,mod2,mod3,mod4,mod5,mod6,mod7,mod8]
mod_idx=-1

for mod in mods:
  mod_idx+=1

  Site_in_this_model=mod.Site_Name.unique() # for the res table

  m=mod.drop(["Unnamed: 0",'Unnamed: 0.1',"Date","Model_number","Site_Name"],axis=1)
  y_isrealis=m.Israelis_Count
  y_tourists=m.Tourists_Count
  y_total=m.Total

  targets_list=[y_isrealis,y_tourists,y_total]
  targets_list_name=["Israelis_Count",'Tourists_Count',"Total"]
  idx=0

  for y in targets_list:

        ############################################################################################################################
        acc_rate = [] 
        for k in range(1,74):
            #reset X df
            X = m.drop(["Israelis_Count","Tourists_Count","Total"],axis=1)
          
            #choose the best k feature by f_regrssion
            f_reg_selector = SelectKBest(f_regression, k=k)
            f_reg_selector.fit(X, y)
            f_reg_support = f_reg_selector.get_support()
            f_reg_feature = X.loc[:,f_reg_support].columns.to_list()
            X=X[f_reg_feature]

            #split the data
            X_val_train, X_test, y_val_train, y_test = train_test_split(X, y, test_size=0.2, random_state=312148513)
            X_train, X_val, y_train, y_val = train_test_split(X_val_train, y_val_train, test_size=0.2, random_state=316173897)
            train_df = pd.merge(left=X_train, right=y_train, left_index=True, right_index=True)
            val_df = pd.merge(left=X_val, right=y_val, left_index=True, right_index=True)
            test_df = pd.merge(left=X_test, right=y_test, left_index=True, right_index=True)

            lm = LinearRegression()   # define our model using least square method
            lm.fit(X_train,y_train)   # Fit our linear model

            fitted = lm.predict(X_val)
            predicted_test = round(pd.Series(fitted, index=y_val.index, name='Predicted_val_'+targets_list_name[idx]),ndigits=2)
            val_df = pd.merge(left=val_df, right=predicted_test, left_index=True, right_index=True)

            correct_rows = val_df.loc[abs(val_df['Predicted_val_'+targets_list_name[idx]]-val_df[targets_list_name[idx]])<=50].any(axis=1).count()
            ACC_val = round(correct_rows/len(test_df),3)
            acc_rate.append(ACC_val)
        ############################################################################################################################
          
          # ## show the most acc k mode feature
        print(np.amax(acc_rate))
        print(np.where(acc_rate == np.amax(acc_rate))[0][0]+1)
        
        trace = go.Scatter(
            x=[i for i in range(1,76)],
            y=acc_rate,
            mode='markers+lines', 
        )
        fig = go.Figure(trace)
        fig.update_layout(
            title='Acc by K Value for mod'+str(mod_idx)+" target "+targets_list_name[idx], 
            xaxis_title='k neighboors',
            yaxis_title='acc rate'
        
        )
        fig.show()

        ############################################################################################################################



        #get k that give the highest acc and start all over
        k = np.where(acc_rate == np.amax(acc_rate))[0][0]+1
        X = m.drop(["Israelis_Count","Tourists_Count","Total"],axis=1)
      
        #choose the best k feature by f_regrssion
        f_reg_selector = SelectKBest(f_regression, k=k)
        f_reg_selector.fit(X, y)
        f_reg_support = f_reg_selector.get_support()
        f_reg_feature = X.loc[:,f_reg_support].columns.to_list()
        X=X[f_reg_feature]

        Descripton ="feature selection by f_regression k="+str(k)+" and the features are:"+str(f_reg_feature)
        #split the data
        X_val_train, X_test, y_val_train, y_test = train_test_split(X, y, test_size=0.2, random_state=312148513)
        X_train, X_val, y_train, y_val = train_test_split(X_val_train, y_val_train, test_size=0.2, random_state=316173897)
        train_df = pd.merge(left=X_train, right=y_train, left_index=True, right_index=True)
        val_df = pd.merge(left=X_val, right=y_val, left_index=True, right_index=True)
        test_df = pd.merge(left=X_test, right=y_test, left_index=True, right_index=True)
        lm = LinearRegression()  # define our model using least square method
        lm.fit(X_train,y_train)   # Fit our linear model


        #train
        fitted = lm.predict(X_train)
        predicted_train = round(pd.Series(fitted, index=y_train.index, name='Predicted_train_'+targets_list_name[idx]),ndigits=2)
        predicted_train
        train_df = pd.merge(left=train_df, right=predicted_train, left_index=True, right_index=True)
        train_df



        #test
        fitted = lm.predict(X_test)
        predicted_test = round(pd.Series(fitted, index=y_test.index, name='Predicted_test_'+targets_list_name[idx]),ndigits=2)
        predicted_test
        test_df = pd.merge(left=test_df, right=predicted_test, left_index=True, right_index=True)
        test_df


        #calculate the residuals
        test_df['residuals'] = test_df['Predicted_test_'+targets_list_name[idx]] - test_df[targets_list_name[idx]]
        train_df['residuals'] = train_df['Predicted_train_'+targets_list_name[idx]] - train_df[targets_list_name[idx]]

        #plot the residuals graph

        fig= go.Figure()
        fig.add_trace(
          go.Scatter(
              x=train_df['Predicted_train_'+targets_list_name[idx]],
              y=train_df.residuals,
              mode='markers',
              name='train residuals',
              marker_color='blue',
              marker_size=1.5,
              marker_line_width=0,

          )
        )
        fig.add_trace(
          go.Scatter(
              x=test_df['Predicted_test_'+targets_list_name[idx]],
              y=test_df.residuals,
              mode='markers',
              name='test residuals',
              marker_color='red',
              marker_size=1.5,
              marker_line_width=0,

          )
        )

        fig.add_trace(
          go.Scatter(
              x=test_df['Predicted_test_'+targets_list_name[idx]],
              y=test_df.residuals*0,
              mode='lines',
              name='zero line',
              marker_color='black',
              marker_size=1.5,
              marker_line_width=0,

          )
        )
        fig.update_layout(
            title="Residuals of Predicted "+targets_list_name[idx]+" mod"+str(mod_idx),
            xaxis_title="Predicted "+targets_list_name[idx],
            yaxis_title="Residuals",
            font=dict(
                size=14,
                color="RebeccaPurple"
            )
        )
        fig.show()
        import os
        mod_str="mod"+str(mod_idx)
        if not os.path.exists(mod_str):
          os.mkdir(mod_str)
        fig.write_image(mod_str+"/results_"+targets_list_name[idx]+'_'+mod_str+'.png', width=1500, height=600)
        #res!

        from sklearn import metrics
        f = open(mod_str+"/results_"+targets_list_name[idx]+'_'+mod_str+'.txt', 'w')
        f.write(Descripton)
         #print(train.columns)
        correct_rows = train_df.loc[abs(train_df['Predicted_train_'+targets_list_name[idx]]-train_df[targets_list_name[idx]])<=50].any(axis=1).count()
        ACC_Training = round(correct_rows/len(train_df),3)

        MSE_Training = metrics.mean_squared_error(train_df[targets_list_name[idx]], train_df['Predicted_train_'+targets_list_name[idx]])
        RMSE_Training = np.sqrt(metrics.mean_squared_error(train_df[targets_list_name[idx]], train_df['Predicted_train_'+targets_list_name[idx]]))
        MAE_Training = metrics.mean_absolute_error(train_df[targets_list_name[idx]], train_df['Predicted_train_'+targets_list_name[idx]])
        R2_Training=metrics.r2_score(train_df[targets_list_name[idx]], train_df['Predicted_train_'+targets_list_name[idx]])

        f.write("------ TRAIN DATA ------\n")
        f.write("Model "+mod_str +" Accuracy: "+str(ACC_Training)+"\n")
        f.write("MSE : "+str(MSE_Training)+", RMSE: "+str(RMSE_Training)+", MAE : "+str(MAE_Training)+"\n")
        f.write("R2 TRAIN "+ str(R2_Training)+"\n")

        correct_rows = test_df.loc[abs(test_df['Predicted_test_'+targets_list_name[idx]]-test_df[targets_list_name[idx]])<=50].any(axis=1).count()
        ACC_Test = round(correct_rows/len(test_df),3)

        MSE_Test = metrics.mean_squared_error(test_df[targets_list_name[idx]], test_df['Predicted_test_'+targets_list_name[idx]])
        RMSE_Test = np.sqrt(metrics.mean_squared_error(test_df[targets_list_name[idx]], test_df['Predicted_test_'+targets_list_name[idx]]))
        MAE_Test = metrics.mean_absolute_error(test_df[targets_list_name[idx]], test_df['Predicted_test_'+targets_list_name[idx]])
        R2_Test = metrics.r2_score(test_df[targets_list_name[idx]], test_df['Predicted_test_'+targets_list_name[idx]])
        f.write("\n")
        f.write("------ TEST DATA ------\n")
        f.write("Model "+mod_str +" Accuracy: "+str(ACC_Test)+"\n")
        f.write("MSE : "+str(MSE_Test)+", RMSE: "+str(RMSE_Test)+", MAE : "+str(MAE_Test)+"\n")
        f.write("R2 TEST "+ str(R2_Test)+"\n")
        f.write("--------------------------------\n")
        f.close()

        new_row={'Descripton':Descripton,"Target":targets_list_name[idx],'Model_type':Model_type,'Model_number':mod_idx,
         'Site_in_this_model':Site_in_this_model,'ACC_Training':ACC_Training,'MAE_Training':MAE_Training,'MSE_Training':MSE_Training,
         'RMSE_Training':RMSE_Training,'R2_Training':R2_Training,'ACC_Test':ACC_Test,'MAE_Test':MAE_Test,'MSE_Test':MSE_Test,'RMSE_Test':RMSE_Test,'R2_Test':R2_Test}
        res=res.append(new_row,ignore_index=True)   
        idx+=1


# res.to_excel("../../../../res.xlsx",index=False)



0.327
12


0.151
5


0.135
7


0.116
5


0.551
3


...

0.65
1


0.209
40


0.164
2


0.8
1


0.164
2
